# Test `semantic_api()`

In [65]:
import pandas as pd
from datetime import date
from collections import Counter

from startupjh import utils
from startupjh import plots
from startupjh.data_collection import consolidated_df
from startupjh.data_preprocessing import data_preprocess
from startupjh.data_preprocessing import data_cleaning, data_enrichment
from startupjh.data_collection import semantic_api

import plotly.graph_objs as go
import plotly.express as px

%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
results_df, all_references_df, total_results, query = semantic_api.get_all_results_from_semantic_scholar()

Enter key words: mars biosignature


In [55]:
plots.make_access_pie(results_df, 'semantic_scholar')

In [57]:
plots.make_pub_per_year(results_df, 'semantic_scholar')

In [58]:
plots.make_citations_per_year(results_df, 'semantic_scholar')

In [60]:
query

'mars biosignature'

In [61]:
plots.make_top_key_words(results_df, query)

In [62]:
results_df

,paperId,url,title,abstract,venue,year,referenceCount,citationCount,influentialCitationCount,isOpenAccess,fieldsOfStudy,authors,key_words
0,1935abaebef93ceb4b18eaf67c12618a001b26bb,https://www.semanticscholar.org/paper/1935abae...,TRACING HOT-SPRING FACIES AND THEIR GEOTHERMAL...,TEXTURES INTO THE TERRESTRIAL GEOLOGIC RECORD:...,,2016,0,1,1,False,None,"[{'authorId': '4662741', 'name': 'K. Campbell'...","[tracing, hotspring, facies, geothermally, sil..."
1,958b808c4e32488bf9f484c7de0b928ddfe94cff,https://www.semanticscholar.org/paper/958b808c...,Preservation of martian organic and environmen...,The Mars Science Laboratory (MSL) has an instr...,Astrobiology,2011,117,215,7,True,"[Environmental Science, Medicine, Geology]","[{'authorId': '4945458', 'name': 'R. Summons'}...","[preservation, martian, organic, environmental..."
2,61d144bd68693abd0ceafd1275969e8596b8c34b,https://www.semanticscholar.org/paper/61d144bd...,Mars Biosignature - Detection Capabilities: A ...,A Mars sample-return mission has been proposed...,,2013,7,3,0,True,[Environmental Science],"[{'authorId': '1751448', 'name': 'C. Weisbin'}...","[mars, biosignature, detection, capabilities, ..."
3,fc7a04d99ce2f9bc8a9c177e6a6cdbebe842641f,https://www.semanticscholar.org/paper/fc7a04d9...,Tracing Hot-Spring Facies and thier Geothermal...,None,,2016,0,0,0,False,[Geology],"[{'authorId': '4662741', 'name': 'K. Campbell'...","[tracing, hotspring, facies, thier, geothermal..."
4,f1376f87575a361568df4b3d891b5e6388519cfd,https://www.semanticscholar.org/paper/f1376f87...,Biosignature Analysis of Mars Soil Analogs fro...,The detection of biosignatures on Mars is of o...,Astrobiology,2020,90,6,0,True,"[Medicine, Environmental Science]","[{'authorId': '32793236', 'name': 'Joost W. Ae...","[biosignature, analysis, mars, soil, analogs, ..."
5,b89e9e58001cf455b55ba9b22d9a1273b20fd28f,https://www.semanticscholar.org/paper/b89e9e58...,Elemental Relics - A Trace Element Biosignatur...,Introduction: The burden of proof for confirmi...,,2020,0,0,0,False,[Geology],"[{'authorId': '8646428', 'name': 'A. Gangidine...","[elemental, relics, trace, element, biosignatu..."
6,15f56931cbfc6a0e2296be1cb9fd4d8fe4217781,https://www.semanticscholar.org/paper/15f56931...,Radiation Environment and Doses on Mars at Oxi...,None,,2020,109,4,0,True,[Geology],"[{'authorId': '13634608', 'name': 'F. Da Pieve...","[radiation, environment, doses, mars, oxia, pl..."
7,bba45399bbdadc942752ca0edb884a0d87675c9f,https://www.semanticscholar.org/paper/bba45399...,Protocol for Biosignature Identification in We...,Introduction: The NASA Mars 2020 rover key obj...,,2020,0,0,0,False,[Geology],"[{'authorId': '5752254', 'name': 'M. Nachon'},...","[protocol, biosignature, identification, wet, ..."
8,73e782e5532960f9c54cf2219ffdff0afbe5c308,https://www.semanticscholar.org/paper/73e782e5...,Biosignature detection by Mars rover equivalen...,Abstract This work details the laboratory anal...,Planetary and Space Science,2019,88,10,0,True,[Geology],"[{'authorId': '145729540', 'name': 'J. Strombe...","[biosignature, detection, mars, rover, equival..."
9,e9040c638702931f009145cf6126874672585e52,https://www.semanticscholar.org/paper/e9040c63...,Metabolic Processes Preserved as Biosignatures...,Abstract Iron-oxidizing bacteria occupy a dist...,Astrobiology,2018,58,6,1,True,"[Chemistry, Medicine]","[{'authorId': '34897565', 'name': 'M. Floyd'},...","[metabolic, processes, preserved, biosignature..."


In [64]:
df, query = consolidated_df.get_consolidated_df()

Enter key words: mars biosignature


NameError: name 'data_cleaning' is not defined

In [66]:
df = data_cleaning.clean_df(df)
df = data_enrichment.get_citation_count(df)

In [67]:
df

,title,journal_is_oa,journal_name,published_date,publisher,authors,key_words,citation_count,published_year
0,On biosignatures for Mars,False,International Journal of Astrobiology,2021-11-18,Cambridge University Press (CUP),"[Frances Westall, Keyron Hickman-Lewis, Barbar...","(biosignatures, mars)",1.0,2021.0
1,A Review of Current Approaches for UAV Autonom...,False,2020 IEEE Aerospace Conference,2020-03-01,IEEE,"[Julian Galvez Serna, Fernando Vanegas, Felipe...","(review, current, approaches, uav, autonomous,...",7.0,2020.0
2,Preservation of Martian Organic and Environmen...,False,Astrobiology,2011-03-01,Mary Ann Liebert Inc,"[Roger E. Summons, Jan P. Amend, David Bish, R...","(preservation, martian, organic, environmental...",227.0,2011.0
3,Hunting for biosignatures on Mars,False,Astronomy & Geophysics,2021-08-01,Oxford University Press (OUP),"[Nisha K Ramkissoon, Mark J Burchell, Peter Fa...","(hunting, biosignatures, mars)",0.0,2021.0
4,False biosignatures on Mars: anticipating ambi...,False,Journal of the Geological Society,2021-11-16,Geological Society of London,"[Sean McMahon, Julie Cosmidis]","(false, biosignatures, mars, anticipating, amb...",0.0,2021.0
...,...,...,...,...,...,...,...,...,...
82,Orbital identification of hydrated silica in J...,no data,"Geophysical Research Letters 46, no.",2019-06-01,no data,"[J. D. Tarnas, J. F. Mustard, Honglei Lin, ...","(orbital, identification, hydrated, silica, je...",43.0,2019.0
83,Slow degradation of ATP in simulated martian e...,no data,"Icarus 194, no.",2008-06-01,no data,"[Andrew C. Schuerger, Patricia Fajardo-Cavazo...","(slow, degradation, atp, simulated, martian, e...",60.0,2008.0
84,Silica deposits on Mars with features resembli...,no data,"Nature communications 7, no.",2016-06-01,no data,"[Steven W. Ruff, and Jack D. Farmer]","(silica, deposits, mars, features, resembling,...",172.0,2016.0
85,Mars' surface radiation environment measured w...,no data,"science 343, no.",2014-06-01,no data,"[Donald M. Hassler, Cary Zeitlin, Robert F. ...","(mars, surface, radiation, environment, measur...",480.0,2014.0
